# FLEX

| Modulation | Baud | Mod Index | Sym Freq | 
| ---------- | ---- | --------- | -------- |
| 2FSK | 1600 | 5.625 | |
| 2FSK | 3200 | 2.8125 | |
| 4FSK | 3200 | 2.8125 | |
| 4FSK | 6400 | 1.40625 | (-4.8, -1.6, 1.6, 4.8) |

## Frequencies
* 152.6 / 169 MHz
* 309.505 / 310.905 MHz
* 929.362 / 929.387 / 929.538 MHz / 929.612 / 929.662 / 929.937 MHz
* 931.063 / 931.863 / 931.938 MHz

## References
https://www.sigidwiki.com/wiki/FLEX


# import libraries

In [ ]:
import numpy as np
import os
import pylab
from scipy import signal as sigmod

# load signal


In [ ]:
file0 = os.environ["HOME"] + "/data/recordings/20200226_22-07-20__fc_929.5130_MHz_fs_600.0000_ksps.32cf"
signal = np.fromfile(file0, dtype=np.complex64)
fs = 600e3

pylab.specgram(signal, Fs=fs)
pylab.show()

fc_offset = 100.3e3


In [ ]:
# shift signal

bb_sig = signal * np.exp(2j*np.pi*-fc_offset*np.arange(len(signal))/fs)
decim = 20


bb_sig = sigmod.resample_poly(bb_sig, 1, decim)
bb_fs = fs / decim

#segment first burst
bb_sig = bb_sig[int(bb_fs*1.25): int(bb_fs*3)]

pylab.specgram(bb_sig, Fs=bb_fs, NFFT=16, noverlap=4)
pylab.show()



In [ ]:
from kcss.receivers.burst_receiver import BurstReceiver as BR
baud = 1600.
spec = {"baud":baud, "symbol_map":[1,0], "mod_index":9600/baud}
bits = BR.fsk_receiver(bb_sig, bb_fs, spec)
print(bits)
#pylab.plot(bits[3240:3300], '.-')
pylab.figure(figsize=[16,8])
pylab.plot(bits,'s-')
pylab.show()

In [ ]:
# POCSAG FCS
FCS = [0,1,1,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,1,0,1,1,1,0,1,1,0,0,0]

bits2 = 2*np.array(bits).astype(np.int8) - 1
fcs2 = 2* np.array(FCS) - 1
cor = np.convolve(bits2, np.flipud(fcs2))
pylab.subplot(311), pylab.plot(bits2)
pylab.subplot(312), pylab.plot(fcs2)
pylab.subplot(313), pylab.plot(cor)
pylab.show()
print(len(bits2))
print(len(fcs2))
print(max(abs(cor)))
